# Airline AI Assistant

An AI Customer Support assistant for an Airline.

Features:
- Get ticket prices, book flights and check passengers in
- Generate images of destination city
- Talk to user and accept audio requests
- Multi-modal: uses language, image and audio models

Technology:
- OpenAI's GPT-4o-mini, Dall-E-3 and TT-1 models
- Gradio for UI
- Tools / function calling
- ffmpeg for audio responses

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import string
import random

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# Use Ollama instead of OpenAI
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Tools

In [ ]:
# Function to retrieve ticket prices

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

# Function to generate PNR
def generate_random_string(length=4):
    possible_characters = string.ascii_letters + string.digits
    random_string = ''.join(random.choice(possible_characters) for _ in range(length))
    return random_string

# Function to book flight

def book_flight(destination_city, ticket_price, passenger):
    print(f"Tool book_flight called for {passenger} traveling to {destination_city} for {ticket_price}")
    return generate_random_string()

# Function to checkin

def checkin_passenger(confirmation_code, last_name):
    print(f"Tool checkin_passenger called to checkin passenger with last name {last_name} using the PNR {confirmation_code}")
    return True

In [ ]:
get_ticket_price("Berlin")
book_flight("Berlin", "$499", "Bob Jones")
checkin_passenger("GEA6", "Jones")

In [ ]:
# Describe the functions for the LLM:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

booking_function = {
    "name": "book_flight",
    "description": "Book a return ticket to the destination city for a passenger and provide them the confirmation code. Call this whenever you need to book a ticket on a flight, for example when a customer asks 'Book me a ticket to this city' or 'Book a flight'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "ticket_price": {
                "type": "string",
                "description": "The price of a return ticket to the destination city",
            },
            "passenger": {
                "type": "string",
                "description": "The first and last name of the passenger to book a ticket for",
            },
        },
        "required": ["destination_city"],
        "required": ["ticket_price"],
        "required": ["passenger"],
        "additionalProperties": False
    }
}

checkin_function = {
    "name": "checkin_passenger",
    "description": "Checkin a passenger on a booked flight using the confirmation code and last name of the passenger. Call this whenever you need to checkin passenger on a flight they have a ticket for, for example when a customer asks 'Check me in for my flight. My confirmatin code is AF3S and my last name is Jones' or 'I'd like to checkin'",
    "parameters": {
        "type": "object",
        "properties": {
            "confirmation_code": {
                "type": "string",
                "description": "The confirmation code or PNR used to identify the ticket for a flight booking",
            },
            "last_name": {
                "type": "string",
                "description": "The last name of the passenger to checkin on the booked flight",
            },
        },
        "required": ["confirmation_code"],
        "required": ["last_name"],
        "additionalProperties": False
    }
}

In [ ]:
# Include a list of tools:

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": booking_function}, {"type": "function", "function": checkin_function}]

## Use the Tool

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    for tool_call in message.tool_calls:
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
    
        result = call_function(name, args)
        response = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(result)
        }
    return response

def call_function(name, args):
    if name == "get_ticket_price":
        return get_ticket_price(**args)
    if name == "book_flight":
        return book_flight(**args)
    if name == "checkin_passenger":
        return checkin_passenger(**args)

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

# Multi-Modal Version

Use DALL-E-3, the image generation and TTS-1 for text-to-speech.

In [ ]:
# Check ffmpeg is installed
!ffmpeg -version
!ffprobe -version
!ffplay -version

In [ ]:
# Imports for handling images

import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

## Tools for Image Generation and Talking

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

## Airline AI Agent Extended

'Agentic AI' and Agentization techniques used:

1. Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
2. The ability for LLMs to use Tools to give them additional capabilities

In [1]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [2]:
# Gradio code for non-preset Chat interface

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

NameError: name 'gr' is not defined

## To do
- ~~Generate image for destination city~~
- ~~Audio rseponse~~
- ~~Add other Tools to book flight and checkin~~
- Add an Agent that translates all responses to a different language and shows on the right hand side, using a different Frontier model, e.g. Claude
- Add an Agent that can listen for Audio and convert it to text